In [5]:
import pandas as pd
import numpy as np
import json
from itertools import product
import datetime

from ib_insync import *

util.startLoop()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

market = 'snp'
# ...variables initialization
with open('var.json', 'r') as fp:
    data = json.load(fp)

host = data['common']['host']
port = data[market]['port']
cid = 1

symbol = 'FDX'

with IB().connect(host=host, port=port, clientId=cid) as ib:
    c = ib.qualifyContracts(Stock(symbol, 'SMART', 'USD'))[0] # ASYNC THIS
    
    # ASYNC THIS
    ohlc = ib.reqHistoricalData(contract=c, endDateTime = '',
                       durationStr='365 D', barSizeSetting= '1 day',
                       whatToShow = 'Trades', useRTH=True)
    
    # reverse sort to have latest date on top
    df_ohlc = util.df(ohlc).sort_index(ascending=False).reset_index(drop=True)
    
    df_ohlc.insert(0, 'symbol', c.symbol)

    df_ohlc['rise'] = [df_ohlc['close'].rolling(i).apply(lambda x: x[0]-x[-1], raw=True).max()
                       for i in range(1, len(df_ohlc)+1)]
    
    df_ohlc['rise'] = df_ohlc['rise'].abs()

    df_ohlc['fall'] = [df_ohlc['close'].rolling(i).apply(lambda x: x[0]-x[-1], raw=True).min()
                       for i in range(1, len(df_ohlc)+1)]
    df_ohlc.fall = df_ohlc.fall.abs()

    df_ohlc = df_ohlc.assign(sd=df_ohlc.close.expanding(1).std(ddof=0))
    df_ohlc.sd = df_ohlc.sd.expanding(1).max() # roll the standard deviation upwards

    undPrice = {c.symbol: ib.reqTickers(c)[0].marketPrice()} # ASYNC THIS
    

    # ASYNC THIS
    chains = {c.symbol: ib.reqSecDefOptParams(
        underlyingSymbol=c.symbol, futFopExchange='',
        underlyingSecType=c.secType, underlyingConId=c.conId)[0]}
    sek = {b for a in [list(product([k], m.expirations, m.strikes))
                       for k, m in chains.items()] for b in a}

    dfc = pd.DataFrame(list(sek), columns=['symbol', 'expiry', 'strike'])
    dfc = dfc.assign(dte=[(util.parseIBDatetime(
        dt)-datetime.datetime.now().date()).days for dt in dfc.expiry])    
    dfc = dfc[dfc.dte <= data['common']['maxdte']] # Limit to max and min dte
    dfc = dfc[dfc.dte >= data['common']['mindte']]
    dfc = dfc.join(dfc.dte.apply(lambda x: df_ohlc.iloc[x][['rise', 'fall', 'sd']])) # integrate rise, fall and stdev

    # remove the calls and puts whose strike is in the threshold of st dev
    dfc['undPrice'] = undPrice[c.symbol]
    dfc = dfc.assign(right=np.where(dfc.strike >= dfc.undPrice, 'C', 'P'))
    c_mask = (dfc.right == 'C') & (dfc.strike > dfc.undPrice + data['common']['callstdmult']*dfc.sd)
    p_mask = (dfc.right == 'P') & (dfc.strike < dfc.undPrice - data['common']['putstdmult']*dfc.sd)
    dfc = dfc[c_mask | p_mask].reset_index(drop=True)

    # Based on filter selection in json weed out...
    dfc = dfc.assign(strikeRef = np.where(dfc.right == 'P', 
                                          dfc.undPrice-dfc.fall, 
                                          dfc.undPrice+dfc.rise))

    if data['common']['callRise']:
        dfc = dfc[~((dfc.right == 'C') & (dfc.strike < dfc.strikeRef))].reset_index(drop=True)

    if data['common']['putFall']:
        dfc = dfc[~((dfc.right =='P') & (dfc.strike > dfc.strikeRef))].reset_index(drop=True)

    if data['common']['onlyPuts']:
        dfc = dfc[dfc.right == 'P'].reset_index(drop=True)

    # limit to nBands
    nBand = data['common']['nBand']
    gb = dfc.groupby(['right'])

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, True, True])
        df_calls = df_calls.groupby(['symbol', 'dte']).head(nBand)
    else:
        df_calls = pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, True, False])
        df_puts = df_puts.groupby(['symbol', 'dte']).head(nBand)
    else:
        df_puts =  pd.DataFrame([])

    dfc = pd.concat([df_puts, df_calls]).reset_index(drop=True)

    # qualify the options
    opts = [Option(i.symbol, i.expiry, i.strike, i.right, data[market]['exchange']) for i in dfc[['symbol', 'expiry', 'strike', 'right']].itertuples()]
    qual_opts = ib.qualifyContracts(*opts)
    

In [11]:
with IB().connect(host=host, port=port, clientId=cid) as ib:


Started to throttle requests
Error 200, reqId 1190936: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200117', strike=132.0, right='P', exchange='SMART')
Error 200, reqId 1190937: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200117', strike=131.0, right='P', exchange='SMART')
Error 200, reqId 1190935: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200110', strike=137.0, right='P', exchange='SMART')
Error 200, reqId 1190940: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200124', strike=129.0, right='P', exchange='SMART')
Error 200, reqId 1190941: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200124', strike=128.0, right='P', exchange=

In [17]:
dfc.head()

,symbol,expiry,strike,dte,rise,fall,sd,undPrice,right,strikeRef
0,FDX,20191213,141.0,7,20.24,42.86,3.860035,153.29,P,110.43
1,FDX,20191213,140.0,7,20.24,42.86,3.860035,153.29,P,110.43
2,FDX,20191213,139.0,7,20.24,42.86,3.860035,153.29,P,110.43
3,FDX,20191220,141.0,14,24.88,77.34,3.860035,153.29,P,75.95
4,FDX,20191220,140.0,14,24.88,77.34,3.860035,153.29,P,75.95


In [32]:
df = util.df(qual_opts).iloc[:, 0:6]

In [33]:
df

,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right
0,OPT,389803517,FDX,20191213,141.0,P
1,OPT,389803506,FDX,20191213,140.0,P
2,OPT,389803491,FDX,20191213,139.0,P
3,OPT,394869462,FDX,20191220,141.0,P
4,OPT,388155860,FDX,20191220,140.0,P
5,OPT,394869459,FDX,20191220,139.0,P
6,OPT,391429474,FDX,20191227,140.0,P
7,OPT,394200324,FDX,20191227,139.0,P
8,OPT,394200309,FDX,20191227,138.0,P
9,OPT,392654258,FDX,20200103,140.0,P
